# Model 2: Implementation

In [1]:
import numpy as np
import matplotlib as plt

In [2]:
# Residual for single point
def r(A, b, zi, wi):
    x = zi.T @ A @zi + b.T @ zi - 1
    return np.maximum(wi * x, 0)

# Residual vector for all points
def R(A, b, Z, W):
    R = np.zeros_like(W)
    for i in range(R.size):
        R[i] = r(A, b, Z[i], W[i])
    return R

#Objective function
def f(x, Z, W):
    m, n = Z.shape
    A = np.zeros((n, n))
    b = np.zeros(n)
    update_A(A, x)
    update_b(b, x)
    print(A)
    print(R(A, b, Z, W))
    return np.sum(R(A, b, Z, W)**2)

# Gradient of objective function
def df(x, Z, W):
    m, n = Z.shape
    A = np.zeros((n, n))
    b = np.zeros(n)
    update_A(A, x)
    update_b(b, x)
    print(R(A, b, Z, W))
    print(jacobian_R(A,b, Z))
    return jacobian_R(A, b, Z).T @ R(A, b, Z, W)

In [3]:
# Insert coefficients in (nxn) matrix A, given (m+k) vector x
def update_A(A, x):
    n = A.shape[0]
    k = n*(n+1)//2
    a = x[:k]
    
    end = 0
    for j in range(n):
        start = end
        end = start + n-j
        A[j, j:] = x[start:end]
        A[j+1:,j] = A[j, j+1:]
    
    return A

# Insert coefficients in (n) centering vector given (n+k) vector x
def update_b(b, x):
    n = b.size
    b = x[-n:]
    return b

# Get the first (k) coefficients of (n+k) vector x, given an (nxn) matrix A
def extract_a(A):
    n = A.shape[0]
    k = n*(n+1)//2
    a = np.zeros(k)
    end = 0
    
    for j in range(n):
        start = end
        end = start + n-j
        a[start:end] = A[j, j:]
        
    return a

In [4]:
# return partial derivatives of residual i, wrt. the matrix coefficients
def r_delA(A, b, zi):
    n = b.size
    rdA = 2 * (zi * (zi * np.ones_like(A)).T)
    rdA = rdA - 1/2 * np.identity(n) * rdA
    return extract_a(rdA)

# Return vector of all partial derivatives of residual i
def del_r(A, b, zi):
    n = b.size
    k = n*(n+1)//2
    dr = np.zeros(k+n)
    dr[:k] = r_delA(A, b, zi)
    dr[k:] = zi
    return dr

#Jacobian of residual vector
def jacobian_R(A, b, z):
    m, n = z.shape
    k = n*(n+1)//2
    R = np.zeros((m, k+n))
    for i in range(m):
        R[i] = del_r(A, b, z[i])
    return R

In [5]:
Z = np.array((2, 3, 2, 4, 4, 5)).reshape(3, 2)
W = np.ones(3)
x = np.array((1, 0, 0, 4, 1, 0))
f(x, Z, W)

[[ 1.  0.]
 [ 0.  0.]]
[  3.   3.  15.]


243.0

In [6]:
df(x, Z, W)

[  3.   3.  15.]
[[  4.  12.   9.   2.   3.]
 [  4.  16.  16.   2.   4.]
 [ 16.  40.  25.   4.   5.]]


array([ 264.,  684.,  450.,   72.,   96.])